In [1478]:
import numpy as np
# import itertools
import random

In [1479]:

def create_hamming_matrix(r: int) -> np.ndarray:
    n = 2**r - 1
    H = np.zeros((r, n), dtype=int)
    
    col = 0
    for i in range(1, n + 1):
        if i & (i - 1):  
            binary = [int(bit) for bit in bin(i)[2:].zfill(r)]
            H[:, col] = binary
            col += 1
    
    for i in range(r):
        H[i, n - r + i] = 1
    
    return H



def create_generator_matrix(H: np.ndarray) -> np.ndarray:
    r, n = H.shape
    k = n - r
    P = H[:, :k]
    I_k = np.eye(k, dtype=int)
    G = np.hstack((I_k, P.T))
    return G



def generate_syndrome_table(H):
    n = H.shape[1]
    syndrome_table = {}
    
    for i in range(n):
        error_vector = np.zeros(n, dtype=int)
        error_vector[i] = 1
        syndrome = tuple(int(s) for s in (H @ error_vector) % 2)
        syndrome_table[syndrome] = (i,)

    return syndrome_table






In [1480]:

def encode_message(message, G):
    return (message @ G) % 2

def introduce_error(codeword, num_errors):
    error_positions = random.sample(range(len(codeword)), num_errors)
    error_vector = np.zeros_like(codeword)
    error_vector[error_positions] = 1
    return (codeword + error_vector) % 2, error_vector

def decode_message(received, H, syndrome_table, is_extended=False):
    syndrome = tuple(int(s) for s in (H @ received) % 2)
    
    if is_extended:
        parity_check = np.sum(received) % 2
        if parity_check == 0 and syndrome == tuple(0 for _ in range(len(syndrome))):
            return received, False  # Ошибок не обнаружено
        elif parity_check == 1 and syndrome in syndrome_table:
            correction_indices = syndrome_table[syndrome]
            corrected = received.copy()
            for index in correction_indices:
                corrected[index] ^= 1  # Исправляем ошибку
            return corrected, True
        else:
            return received, False  # Обнаружена неисправимая ошибка
    else:
        if syndrome == tuple(0 for _ in range(len(syndrome))):
            return received, False  # Ошибок не обнаружено
        if syndrome in syndrome_table:
            error_position = syndrome_table[syndrome][0]
            corrected = received.copy()
            corrected[error_position] ^= 1  # Исправляем ошибку
            return corrected, True
        return received, False  # Неисправимая ошибка

def hamming_code_test(r, H, G, syndrome_table, num_errors, is_extended=False):
    k = G.shape[0]  
    message = np.random.randint(0, 2, k)
    
    print(f"Тест для r={r}, количество ошибок={num_errors}")
    print(f"Исходное сообщение (u): {message}")
    
    codeword = encode_message(message, G)
    print(f"Кодовое слово (v): {codeword}")
    
    received, error = introduce_error(codeword, num_errors)
    print(f"Допущенная ошибка: {error}")
    print(f"Принятое с ошибкой слово: {received}")
    
    syndrome = (H @ received) % 2
    print(f"Синдром принятого сообщения: {syndrome}")
    
    decoded, corrected = decode_message(received, H, syndrome_table, is_extended)

    if corrected:
        print(f"Исправленное сообщение: {[int(bit) for bit in decoded]}")
        if np.array_equal(decoded, codeword):
            print("Ошибка была исправлена успешно!")
        else:
            print("Ошибка не была исправлена корректно.")
    elif np.array_equal(syndrome, np.zeros_like(syndrome)):
        print("Ошибок не обнаружено.")
    else:
        print("Обнаружена неисправимая ошибка.")
    
    print("\n")






In [1481]:
r = 2  
n = 2**r - 1
k = n - r
H = create_hamming_matrix(r)
print("Проверочная матрица H:")
print(H.T)
G = create_generator_matrix(H)
print("Порождающая матрица G:")
print(G)
syndrome_table = generate_syndrome_table(H)
print("Таблица синдромов:")
print(syndrome_table)

Проверочная матрица H:
[[1 1]
 [1 0]
 [0 1]]
Порождающая матрица G:
[[1 1 1]]
Таблица синдромов:
{(1, 1): (0,), (1, 0): (1,), (0, 1): (2,)}


In [1482]:
hamming_code_test(r, H,G,syndrome_table,1,is_extended=False)
hamming_code_test(r, H,G,syndrome_table,2,is_extended=False)

Тест для r=2, количество ошибок=1
Исходное сообщение (u): [0]
Кодовое слово (v): [0 0 0]
Допущенная ошибка: [0 1 0]
Принятое с ошибкой слово: [0 1 0]
Синдром принятого сообщения: [1 0]
Исправленное сообщение: [0, 0, 0]
Ошибка была исправлена успешно!


Тест для r=2, количество ошибок=2
Исходное сообщение (u): [0]
Кодовое слово (v): [0 0 0]
Допущенная ошибка: [1 1 0]
Принятое с ошибкой слово: [1 1 0]
Синдром принятого сообщения: [0 1]
Исправленное сообщение: [1, 1, 1]
Ошибка не была исправлена корректно.




In [1483]:
hamming_code_test(r, H,G,syndrome_table,3,is_extended=False)

Тест для r=2, количество ошибок=3
Исходное сообщение (u): [1]
Кодовое слово (v): [1 1 1]
Допущенная ошибка: [1 1 1]
Принятое с ошибкой слово: [0 0 0]
Синдром принятого сообщения: [0 0]
Ошибок не обнаружено.




In [1484]:
r = 3  
H = create_hamming_matrix(r)
print("Проверочная матрица H:")
print(H.T)
G = create_generator_matrix(H)
print("Порождающая матрица G:")
print(G)
syndrome_table = generate_syndrome_table(H)

Проверочная матрица H:
[[0 1 1]
 [1 0 1]
 [1 1 0]
 [1 1 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]]
Порождающая матрица G:
[[1 0 0 0 0 1 1]
 [0 1 0 0 1 0 1]
 [0 0 1 0 1 1 0]
 [0 0 0 1 1 1 1]]


In [1485]:
hamming_code_test(r, H,G,syndrome_table,1,is_extended=False)
hamming_code_test(r, H,G,syndrome_table,2,is_extended=False)

Тест для r=3, количество ошибок=1
Исходное сообщение (u): [1 0 1 0]
Кодовое слово (v): [1 0 1 0 1 0 1]
Допущенная ошибка: [0 0 0 0 1 0 0]
Принятое с ошибкой слово: [1 0 1 0 0 0 1]
Синдром принятого сообщения: [1 0 0]
Исправленное сообщение: [1, 0, 1, 0, 1, 0, 1]
Ошибка была исправлена успешно!


Тест для r=3, количество ошибок=2
Исходное сообщение (u): [1 0 0 0]
Кодовое слово (v): [1 0 0 0 0 1 1]
Допущенная ошибка: [0 0 1 0 0 0 1]
Принятое с ошибкой слово: [1 0 1 0 0 1 0]
Синдром принятого сообщения: [1 1 1]
Исправленное сообщение: [1, 0, 1, 1, 0, 1, 0]
Ошибка не была исправлена корректно.




In [1486]:
hamming_code_test(r, H,G,syndrome_table,3,is_extended=False)

Тест для r=3, количество ошибок=3
Исходное сообщение (u): [1 1 1 1]
Кодовое слово (v): [1 1 1 1 1 1 1]
Допущенная ошибка: [0 0 0 1 0 1 1]
Принятое с ошибкой слово: [1 1 1 0 1 0 0]
Синдром принятого сообщения: [1 0 0]
Исправленное сообщение: [1, 1, 1, 0, 0, 0, 0]
Ошибка не была исправлена корректно.




In [1487]:
r = 4  
H = create_hamming_matrix(r)
print("Проверочная матрица H:")
print(H.T)
G = create_generator_matrix(H)
print("Порождающая матрица G:")
print(G)
syndrome_table = generate_syndrome_table(H)

Проверочная матрица H:
[[0 0 1 1]
 [0 1 0 1]
 [0 1 1 0]
 [0 1 1 1]
 [1 0 0 1]
 [1 0 1 0]
 [1 0 1 1]
 [1 1 0 0]
 [1 1 0 1]
 [1 1 1 0]
 [1 1 1 1]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]
Порождающая матрица G:
[[1 0 0 0 0 0 0 0 0 0 0 0 0 1 1]
 [0 1 0 0 0 0 0 0 0 0 0 0 1 0 1]
 [0 0 1 0 0 0 0 0 0 0 0 0 1 1 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 1 0 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 1 0]
 [0 0 0 0 0 0 1 0 0 0 0 1 0 1 1]
 [0 0 0 0 0 0 0 1 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 1 1 0 1]
 [0 0 0 0 0 0 0 0 0 1 0 1 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]]


In [1488]:
hamming_code_test(r, H,G,syndrome_table,1,is_extended=False)
hamming_code_test(r, H,G,syndrome_table,2,is_extended=False)

Тест для r=4, количество ошибок=1
Исходное сообщение (u): [1 1 1 0 1 1 1 1 0 1 0]
Кодовое слово (v): [1 1 1 0 1 1 1 1 0 1 0 1 0 1 0]
Допущенная ошибка: [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Принятое с ошибкой слово: [1 1 1 0 1 0 1 1 0 1 0 1 0 1 0]
Синдром принятого сообщения: [1 0 1 0]
Исправленное сообщение: [1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0]
Ошибка была исправлена успешно!


Тест для r=4, количество ошибок=2
Исходное сообщение (u): [0 0 0 1 0 1 0 1 1 0 1]
Кодовое слово (v): [0 0 0 1 0 1 0 1 1 0 1 0 0 1 1]
Допущенная ошибка: [0 0 1 0 1 0 0 0 0 0 0 0 0 0 0]
Принятое с ошибкой слово: [0 0 1 1 1 1 0 1 1 0 1 0 0 1 1]
Синдром принятого сообщения: [1 1 1 1]
Исправленное сообщение: [0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1]
Ошибка не была исправлена корректно.




In [1489]:
hamming_code_test(r, H,G,syndrome_table,3,is_extended=False)

Тест для r=4, количество ошибок=3
Исходное сообщение (u): [0 0 1 0 0 1 1 0 0 0 1]
Кодовое слово (v): [0 0 1 0 0 1 1 0 0 0 1 1 0 0 0]
Допущенная ошибка: [0 0 0 1 0 0 0 0 0 1 0 0 0 1 0]
Принятое с ошибкой слово: [0 0 1 1 0 1 1 0 0 1 1 1 0 1 0]
Синдром принятого сообщения: [1 0 1 1]
Исправленное сообщение: [0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0]
Ошибка не была исправлена корректно.




In [ ]:


def create_extended_hamming_check_matrix(H: np.ndarray) -> np.ndarray:
    r, n = H.shape
    H_extended = np.zeros((r + 1, n + 1), dtype=int)
    
    H_extended[:-1, :-1] = H  
    H_extended[-1, :-1] = np.sum(H, axis=0) % 2  
    H_extended[:, -1] = 1  
    
    return H_extended

def create_extended_generator_matrix(G: np.ndarray) -> np.ndarray:
    k, n = G.shape
    G_extended = np.zeros((k, n + 1), dtype=int)
    
    G_extended[:, :-1] = G  
    G_extended[:, -1] = np.sum(G, axis=1) % 2  
    
    return G_extended



















In [1491]:
r = 2
H = create_hamming_matrix(r)
G = create_generator_matrix(H)

H_extended = create_extended_hamming_check_matrix(H)

print("\nРасширенная проверочная матрица H:")
print(H_extended.T)

G_extended = create_extended_generator_matrix(G)
print("\nРасширенная порождающая матрица G:")
print(G_extended)
syndrome_table = generate_syndrome_table(H_extended)



Расширенная проверочная матрица H:
[[1 1 0]
 [1 0 1]
 [0 1 1]
 [1 1 1]]

Расширенная порождающая матрица G:
[[1 1 1 1]]


In [1492]:
hamming_code_test(r, H_extended,G_extended,syndrome_table,1,is_extended=True)
hamming_code_test(r, H_extended,G_extended,syndrome_table,2,is_extended=True)
hamming_code_test(r, H_extended,G_extended,syndrome_table,3,is_extended=True)

Тест для r=2, количество ошибок=1
Исходное сообщение (u): [0]
Кодовое слово (v): [0 0 0 0]
Допущенная ошибка: [0 0 1 0]
Принятое с ошибкой слово: [0 0 1 0]
Синдром принятого сообщения: [0 1 1]
Исправленное сообщение: [0, 0, 0, 0]
Ошибка была исправлена успешно!


Тест для r=2, количество ошибок=2
Исходное сообщение (u): [1]
Кодовое слово (v): [1 1 1 1]
Допущенная ошибка: [1 1 0 0]
Принятое с ошибкой слово: [0 0 1 1]
Синдром принятого сообщения: [1 0 0]
Обнаружена неисправимая ошибка.


Тест для r=2, количество ошибок=3
Исходное сообщение (u): [0]
Кодовое слово (v): [0 0 0 0]
Допущенная ошибка: [1 0 1 1]
Принятое с ошибкой слово: [1 0 1 1]
Синдром принятого сообщения: [0 1 0]
Обнаружена неисправимая ошибка.




In [1493]:
r = 3
H = create_hamming_matrix(r)
G = create_generator_matrix(H)


H_extended = create_extended_hamming_check_matrix(H)
print("\nРасширенная проверочная матрица H:")
print(H_extended.T)


G_extended = create_extended_generator_matrix(G)
print("\nРасширенная порождающая матрица G:")
print(G_extended)
syndrome_table = generate_syndrome_table(H_extended)



Расширенная проверочная матрица H:
[[0 1 1 0]
 [1 0 1 0]
 [1 1 0 0]
 [1 1 1 1]
 [1 0 0 1]
 [0 1 0 1]
 [0 0 1 1]
 [1 1 1 1]]

Расширенная порождающая матрица G:
[[1 0 0 0 0 1 1 1]
 [0 1 0 0 1 0 1 1]
 [0 0 1 0 1 1 0 1]
 [0 0 0 1 1 1 1 0]]


In [1494]:
hamming_code_test(r, H_extended,G_extended,syndrome_table,1,is_extended=True)
hamming_code_test(r, H_extended,G_extended,syndrome_table,2,is_extended=True)

Тест для r=3, количество ошибок=1
Исходное сообщение (u): [1 0 1 1]
Кодовое слово (v): [1 0 1 1 0 1 0 0]
Допущенная ошибка: [0 0 0 0 0 0 0 1]
Принятое с ошибкой слово: [1 0 1 1 0 1 0 1]
Синдром принятого сообщения: [1 1 1 1]
Исправленное сообщение: [1, 0, 1, 1, 0, 1, 0, 0]
Ошибка была исправлена успешно!


Тест для r=3, количество ошибок=2
Исходное сообщение (u): [0 1 0 1]
Кодовое слово (v): [0 1 0 1 0 1 0 1]
Допущенная ошибка: [1 1 0 0 0 0 0 0]
Принятое с ошибкой слово: [1 0 0 1 0 1 0 1]
Синдром принятого сообщения: [0 0 1 1]
Обнаружена неисправимая ошибка.




In [1495]:
hamming_code_test(r, H_extended,G_extended,syndrome_table,3,is_extended=True)

Тест для r=3, количество ошибок=3
Исходное сообщение (u): [1 1 0 1]
Кодовое слово (v): [1 1 0 1 0 0 1 0]
Допущенная ошибка: [0 0 1 0 0 1 1 0]
Принятое с ошибкой слово: [1 1 1 1 0 1 0 0]
Синдром принятого сообщения: [1 0 1 0]
Исправленное сообщение: [1, 0, 1, 1, 0, 1, 0, 0]
Ошибка не была исправлена корректно.




In [1496]:
r = 4
H = create_hamming_matrix(r)
G = create_generator_matrix(H)

# Создаем расширенную проверочную матрицу
H_extended = create_extended_hamming_check_matrix(H)
print("\nРасширенная проверочная матрица H:")
print(H_extended.T)

# Создаем расширенную порождающую матрицу
G_extended = create_extended_generator_matrix(G)
print("\nРасширенная порождающая матрица G:")
print(G_extended)
syndrome_table = generate_syndrome_table(H_extended)


Расширенная проверочная матрица H:
[[0 0 1 1 0]
 [0 1 0 1 0]
 [0 1 1 0 0]
 [0 1 1 1 1]
 [1 0 0 1 0]
 [1 0 1 0 0]
 [1 0 1 1 1]
 [1 1 0 0 0]
 [1 1 0 1 1]
 [1 1 1 0 1]
 [1 1 1 1 0]
 [1 0 0 0 1]
 [0 1 0 0 1]
 [0 0 1 0 1]
 [0 0 0 1 1]
 [1 1 1 1 1]]

Расширенная порождающая матрица G:
[[1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
 [0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1]
 [0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1]
 [0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0]
 [0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 1]
 [0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0]
 [0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1]
 [0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1]]


In [1497]:
hamming_code_test(r, H_extended,G_extended,syndrome_table,1,is_extended=True)
hamming_code_test(r, H_extended,G_extended,syndrome_table,2,is_extended=True)

Тест для r=4, количество ошибок=1
Исходное сообщение (u): [0 0 1 1 1 1 0 0 1 0 1]
Кодовое слово (v): [0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0]
Допущенная ошибка: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Принятое с ошибкой слово: [0 1 1 1 1 1 0 0 1 0 1 0 0 0 0 0]
Синдром принятого сообщения: [0 1 0 1 0]
Исправленное сообщение: [0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
Ошибка была исправлена успешно!


Тест для r=4, количество ошибок=2
Исходное сообщение (u): [1 1 1 0 1 1 1 0 1 1 0]
Кодовое слово (v): [1 1 1 0 1 1 1 0 1 1 0 1 0 1 1 1]
Допущенная ошибка: [1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
Принятое с ошибкой слово: [0 1 1 0 0 1 1 0 1 1 0 1 0 1 1 1]
Синдром принятого сообщения: [0 1 0 1 1]
Обнаружена неисправимая ошибка.




In [1498]:
hamming_code_test(r, H_extended,G_extended,syndrome_table,3,is_extended=True)

Тест для r=4, количество ошибок=3
Исходное сообщение (u): [0 0 1 0 1 0 0 0 1 0 1]
Кодовое слово (v): [0 0 1 0 1 0 0 0 1 0 1 1 1 0 1 1]
Допущенная ошибка: [1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
Принятое с ошибкой слово: [1 1 1 1 1 0 0 0 1 0 1 1 1 0 1 1]
Синдром принятого сообщения: [1 1 1 0 0]
Обнаружена неисправимая ошибка.


